In [1]:
import pandas as pd
import json
import numpy as np
import yaml

with open('article_publication.json') as f:
    article_publication = json.load(f)
    
df = pd.DataFrame(article_publication)
df = df.fillna(0)
    
with open('config.yaml') as f:
    biases = pd.Series(yaml.load(f)['publication_bias'])
    
publications = biases.sort_values().index
words = list(list(article_publication.values())[0].keys())

In [2]:
# dd = {}
# for i in range(df.shape[0]):
#     corr = np.corrcoef(df.iloc[i], biases)[0][1]
#     dd[df.index[i]] = corr

# with open('corr.json', 'w') as f:
#     json.dump(dd, f)

In [3]:
with open('corr.json') as f:
    dd = json.load(f)

In [4]:
words_filtered = df[(df.sum(axis=1) > 5000)].dropna().index
words_filtered = set(words_filtered)

In [5]:
dd_filtered = {k : v for k, v in dd.items() if k in words_filtered}
k_vs = sorted(dd_filtered.items(), key=lambda k_v: k_v[1], reverse=True)

In [6]:
using_keywords = [k for k, v in k_vs[:50] + k_vs[-50:]]

In [7]:
keyword_max_dict = {keyword: max(df[publication][keyword] for publication in publications)
                    for keyword in using_keywords}

In [8]:
with open('../visualization/d3/data_1.csv', 'w') as f:
    f.write('Media,Bias Level,' + ','.join(using_keywords) + '\n')
    for publication in publications:
        f.write(','.join([publication, str(biases[publication]), *[str(df[publication][keyword] / keyword_max_dict[keyword]) for keyword in using_keywords]]) + '\n')